In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
def get_token(file_path):
    with open(file_path) as f:
        data = f.read()
    token = data.strip()
    return token

my_token = get_token("../docs/ibmq_tokens.txt")
service = QiskitRuntimeService(channel="ibm_quantum", token=my_token)
 
backend = service.least_busy(simulator=False, operational=True)
print(backend)

<IBMBackend('ibm_kyiv')>


In [2]:
import sys
sys.path.append("../")
from qmg.generator import MoleculeGenerator
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# parameter settings for epoxide derivatives
num_heavy_atom = 2
random_seed = 9

num_sample = 100

mg = MoleculeGenerator(num_heavy_atom, backend_name="ibmq", remove_bond_disconnection=False) # qiskit_aer

# smiles_dict, validity, diversity = mg.sample_molecule(num_sample, random_seed = random_seed)
# print(smiles_dict)
# print("Validity: {:.2f}%".format(validity*100))
# print("Diversity: {:.2f}%".format(diversity*100))

Using IBM Quantum backend: <IBMBackend('ibm_kyiv')>
>>> Job ID: cyfzt68cw2k00088xwj0


TypeError: cannot unpack non-iterable NoneType object

In [8]:
print(mg.qc)

       ┌────────────┐                                       ┌─┐              »
q5_0: ─┤ Ry(1.4546) ├───■──────────────────────■────────────┤M├──────────────»
       └───┬───┬────┘ ┌─┴─┐              ┌─────┴──────┐     └╥┘              »
q5_1: ─────┤ X ├──────┤ X ├──────■───────┤ Ry(1.1728) ├──■───╫───────────────»
      ┌────┴───┴────┐ └───┘┌─────┴──────┐└────────────┘┌─┴─┐ ║               »
q5_2: ┤ Ry(0.43523) ├──────┤ Ry(2.7224) ├──────■───────┤ X ├─╫───────■───────»
      ├─────────────┴┐     └────────────┘    ┌─┴─┐     └───┘ ║ ┌─────┴──────┐»
q5_3: ┤ Ry(0.020216) ├───────────────────────┤ X ├───────────╫─┤ Ry(1.5795) ├»
      └──────────────┘                       └───┘           ║ └────────────┘»
q5_4: ───────────────────────────────────────────────────────╫───────────────»
                                                             ║               »
q5_5: ───────────────────────────────────────────────────────╫───────────────»
                                                    

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

def get_token(file_path):
    with open(file_path) as f:
        data = f.read()
    token = data.strip()
    return token

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token=get_token("../docs/ibmq_tokens.txt")
)
job = service.job('cyfzt68cw2k00088xwj0')
job_result = job.result()

RuntimeJobFailureError: 'Unable to retrieve job result. Error code 1500; Internal error. -- Try again or contact support. -- https://ibm.biz/error_codes#1500'

In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.classical import expr
from qiskit import generate_preset_pass_manager

from qiskit_ibm_runtime import SamplerV2 as Sampler

def get_dynamic_CNOT_circuit(num_qubit):
    """
    (1) 1D chain of nearest neighbors
    (2) 0th qubit is the control, and the last qubit (num_qubit-1) is the target
    (3) The control qubit starts in the + state
    """
    num_ancilla = num_qubit - 2
    num_ancilla_pair = int(num_ancilla / 2)

    qr = QuantumRegister(num_qubit)
    cr1 = ClassicalRegister(num_ancilla_pair, name="cr1") # The parity-controlled X gate
    cr2 = ClassicalRegister(num_ancilla - num_ancilla_pair, name="cr2") # The parity-controlled Z gate
    cr3 = ClassicalRegister(2, name="cr3") # For the final measurements on the control and target qubits
    qc = QuantumCircuit(qr, cr1, cr2, cr3)

    # Initialize the control qubit
    qc.h(0)
    qc.barrier()

    # Entangle the contorl qubit and the first ancilla qubit
    qc.cx(0,1)

    # Create Bell pairs on ancilla qubits
    # The first ancilla qubit in index 1
    for i in range(num_ancilla_pair):
        qc.h(2+2*i)
        qc.cx(2+2*i, 2+2*i+1)

    # Prepare Bell pairs on staggered ancilla and data qubits
    for i in range(num_ancilla_pair+1):
        qc.cx(1+2*i, 1+2*i+1)
    for i in range(1, num_ancilla_pair+2):
        qc.h(2*i-1)

    # Measurement on alternating ancilla qubits starting with the first one
    # Keep track of the parity for eventual conditional Z gate
    for i in range(1, num_ancilla_pair+2):
        qc.measure(2*i - 1, cr2[i-1])
        if i == 1:
            parity_control = expr.lift(cr2[i-1])
        else:
            parity_control = expr.bit_xor(cr2[i-1], parity_control)

    # Measurement on staggered alternating ancilla qubits starting with the second
    # Keep track of the parity of eventual conditional X gate
    for i in range(num_ancilla_pair):
        qc.measure(2*i + 2, cr1[i])
        if i == 0:
            parity_target = expr.lift(cr1[i])
        else:
            parity_target = expr.bit_xor(cr1[i], parity_target)

    with qc.if_test(parity_control):
        qc.z(0)

    with qc.if_test(parity_target):
        qc.x(-1)

    # Final measurements on the control and target qubits
    qc.measure(0, cr3[0])
    qc.measure(-1, cr3[1])

    return qc

qc = get_dynamic_CNOT_circuit(num_qubit=7)

my_token = get_token("../docs/ibmq_tokens.txt")
service = QiskitRuntimeService(channel="ibm_quantum", token=my_token)
backend = service.least_busy(simulator=False, operational=True)
print("Using IBM Quantum backend:", backend)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_qc = pm.run(qc)
qc_transpiled_list = pm.run([transpiled_qc])

sampler = Sampler(mode=backend)
sampler.options.default_shots = 100

job = sampler.run(qc_transpiled_list)
print(job.job_id())

Using IBM Quantum backend: <IBMBackend('ibm_sherbrooke')>
cwn8wd140e000089g3p0
